Natural Language Processing - Homework 1

Pham Lan Phuong - 210120

In [ ]:
import re
import nltk
from nltk.util import ngrams
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/lanphgphm/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
with open('testing_data.txt', 'r') as f:
    test = f.read()

with open('training_data.txt', 'r') as f:
    train = f.read()

# Question 1
Find all sentences that contain “to be” verbs (i.e. “is”, “are”, ...) in the training data
file.

In [ ]:
sentences = nltk.tokenize.sent_tokenize(train)
be_sentences = []
for sentence in sentences:
    # use re.search() because we care if the sentence contains 'be' or not
    contains_be = re.search(r'\b(is|are|am|be|being|been|was|were)\b', sentence)
    if contains_be:
        be_sentences.append(sentence)

be_sentences

['As part of the major, students will be equipped with the foundational knowledge in Computer Science and relevant disciplines.',
 'They will be exposed to essential areas of the CS discipline including theory, systems, and applications.',
 'They will learn about the underlying mathematical ideas that are critical for computation, establish proficiency in the process of designing systems and applications, gain experience in collecting and analyzing data using modern technologies, and begin to develop an understanding for the role of users in the design of systems and applications.',
 'The Computer Science major at Fulbright is designed to prepare students for work in industry or continue their lifelong learning as well as potential graduate-level studies.',
 'All students are first required to take the core courses in Liberal Arts and Science.',
 'In addition to the two courses in “Global Humanities and Social Change”, and “Modern Vietnamese Culture and Society”, they will be exposed t

# Question 2
Build a unigram model and a bigram model (both are with add-one smoothing) from the training data file. Then calculate and compare the perplexity score of these two models
on the testing data file.

### Helper functions

In [ ]:
def add_one_smoothing(ngram_count, unique_ngram_size):
    for word in ngram_count:
        ngram_count[word] += 1
        unique_ngram_size += 1
    return ngram_count, unique_ngram_size

In [ ]:
def ngram_probability(ngram_count, unique_ngram_size):
    for word in ngram_count:
        ngram_count[word] = ngram_count[word] / unique_ngram_size
    return ngram_count

In [ ]:
def unigram_ppl(test, unigram_probability):
    test_words = nltk.tokenize.word_tokenize(test)
    test_ppl = 1

    for word in test_words:
        if word in unigram_probability.keys():
            test_ppl *= 1/unigram_probability[word]

    test_ppl = test_ppl ** (1/len(test_words))
    return test_ppl

In [ ]:
def bigram_ppl(test, bigram_probability, unigram_probability):
    test_words = nltk.tokenize.word_tokenize(test)
    test_bigrams = list(ngrams(sequence=test_words, n=2))
    test_ppl = 1

    for pair in test_bigrams:
        context = pair[0]
        if pair in bigram_probability.keys():
            cond_prob = bigram_probability[pair] / unigram_probability[context]
            test_ppl *= 1/cond_prob


    test_ppl = test_ppl ** (1/len(test_words))
    return test_ppl

### Unigram & add-one smoothing

In [ ]:
# create a dictionary counting word frequency of all words in corpus
# create copy of dictionary and add-one smoothing
train_words = nltk.tokenize.word_tokenize(train)

unigram_count = {}
for word in train_words:
    if word in unigram_count.keys():
        unigram_count[word] += 1
    else:
        unigram_count[word] = 1

unigram_size = len(unigram_count)

In [ ]:
from copy import deepcopy
unigram_count_copy = deepcopy(unigram_count)
unigram_size_copy = deepcopy(unigram_size)

smoothed_unigram_count, smoothed_vocab_size = \
    add_one_smoothing(unigram_count_copy, unigram_size_copy)

In [ ]:
smoothed_unigram_probability = ngram_probability(smoothed_unigram_count, smoothed_vocab_size)
unigram_ppl(test, smoothed_unigram_probability)

37.57928459350777

### Bigram & add-one smoothing

In [ ]:
# create a raw bigram count
bigrams = list(ngrams(sequence=train_words, n=2))
bigram_count = {}

for pair in bigrams:
    if pair in bigram_count.keys():
        bigram_count[pair] += 1
    else:
        bigram_count[pair] = 1

bigram_size = len(bigram_count)

In [ ]:
bigram_count_copy = deepcopy(bigram_count)
bigram_size_copy = deepcopy(bigram_size)

smoothed_bigram_count, smoothed_bigram_size = \
    add_one_smoothing(bigram_count_copy, bigram_size_copy)

In [ ]:
smoothed_bigram_probability = ngram_probability(smoothed_bigram_count, smoothed_bigram_size)
bigram_ppl(test, smoothed_bigram_probability, smoothed_unigram_probability)

2.0643605162383167

With the same training corpus, bigram has significantly lower perplexity score of
2.0643605162383167 compared to unigram with perplexity score of 37.57928459350777.

Bigram performs better on test set.

### Intermediate results for Question 2

In [ ]:
key = lambda x: x[1]
sorted_unigram_count = sorted(unigram_count.items(), key=key, reverse=True)
sorted_unigram_count[:20]

[(',', 52),
 ('and', 37),
 ('.', 31),
 ('the', 30),
 ('in', 27),
 ('of', 22),
 ('to', 20),
 ('courses', 14),
 ('will', 12),
 ('Science', 10),
 ('their', 10),
 ('Computer', 9),
 ('students', 9),
 ('for', 9),
 ('major', 8),
 ('a', 7),
 ('science', 7),
 ('with', 6),
 ('knowledge', 6),
 ('an', 5)]

In [ ]:
sorted_bigram_count = sorted(bigram_count.items(), key=key, reverse=True)
sorted_bigram_count[:20]

[((',', 'and'), 12),
 (('Computer', 'Science'), 9),
 (('in', 'the'), 8),
 (('Science', 'major'), 4),
 (('will', 'be'), 4),
 (('knowledge', 'in'), 4),
 (('in', 'Computer'), 4),
 (('areas', 'of'), 4),
 (('and', 'applications'), 4),
 (('science', ','), 4),
 (('computer', 'science'), 4),
 (('courses', 'in'), 4),
 (('”', ','), 4),
 (('of', 'the'), 3),
 (('the', 'major'), 3),
 (('students', 'will'), 3),
 (('with', 'the'), 3),
 (('.', 'They'), 3),
 (('They', 'will'), 3),
 (('applications', '.'), 3)]

In [ ]:
# showing bigram conditional probability,
# code taken from function bigram_ppl() above
# showing top 20 bigrams sorted by joint probability, not showing all because too long
top_20_bigrams = dict(sorted_bigram_count[:20])

for pair in top_20_bigrams.keys():
    context = pair[0]
    if pair in smoothed_bigram_probability.keys():
        cond_prob = smoothed_bigram_probability[pair] / smoothed_unigram_probability[context]
        print(f'P({pair[1]}|{pair[0]}) = ', cond_prob)

P(and|,) =  0.12533920761102066
P(Science|Computer) =  0.5109983079526226
P(the|in) =  0.16424945612762873
P(major|Science) =  0.23227195816028304
P(be|will) =  0.19653781075100873
P(in|knowledge) =  0.36499879139473046
P(Computer|in) =  0.09124969784868261
P(of|areas) =  0.4258319232938522
P(applications|and) =  0.06723661946745035
P(,|science) =  0.3193739424703892
P(science|computer) =  0.5109983079526226
P(in|courses) =  0.1703327693175409
P(,|”) =  0.4258319232938522
P(the|of) =  0.0888692709482822
P(major|the) =  0.0659352655422739
P(will|students) =  0.20439932318104906
P(the|with) =  0.29199903311578435
P(They|.) =  0.06387478849407784
P(will|They) =  0.5109983079526227
P(.|applications) =  0.4087986463620981
